In [1]:
import pandas as pd 
from create_df import load_df, implement_column_creation
pd.set_option('display.max_columns', None)
df = load_df()
df = implement_column_creation(df)
df

/home/sam/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/sam/Documents/DSI/capstone/poker/src/datacleaning/dependent_columns.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prior_actions'] = df.HandHistory.apply(lambda x: make_lst(x))


,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,prior_actions,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,"[{'P18_572249IY': [], 'P14_459713MR': [], 'P25...",0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,"[{'P18_572249IY': [0], 'P14_459713MR': [0], 'P...",320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,"[{'P18_572249IY': [0, 0], 'P14_459713MR': [0, ...",0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,"[{'P18_572249IY': [0, 0, 0], 'P25_871309MO': [...",0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,"[{'P18_572249IY': [0, 0, 0, 0], 'P25_871309MO'...",0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:25,82,41343764,5.25,Holdem NL,200.0,"[HA, H3]",1850.0,Hyper Turbo (500 Chips),700.0,1850.0,2,0.500000,6,100.0,"{'P6_773472PB': 3, 'Hero': 7}","[P6_773472PB, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,7.0,200.0,9.25,-1150.0,1.0,1,0.378378,0,"[{'P6_773472PB': [0, 23, 23, 0, 0, 23, 0, 0, 2...",0.0,0.0,0.0,1650.0,14.0,3.0,1,0,11.0
54300,54334,4286799339,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 08:12:29,82,41343587,5.25,Holdem NL,100.0,"[D6, H4]",196.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P1_574120EG, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,3.0,200.0,0.98,-100.0,0.0,0,0.489796,0,"[{'P1_574120EG': [0, 0, 23, 0, 0, 0, 23, 0, 0,...",0.0,0.0,0.0,0.0,6.0,4.0,0,0,2.0
54301,54335,4286799383,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:34,82,41343764,5.25,Holdem NL,100.0,"[C8, H6]",700.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P

In [8]:
def prior_actions_for_stats_trying_something_else(df):
    '''
    finds actions of player prior to the hand currently being played 
    '''
    mask = df['TournamentNumber'] != '' # ignores cash hands for time being, make this optional w/ argument? 
    df = df[mask]
    unique_TN = df['TournamentNumber'].unique()
    df['prior_actions'] = df.HandHistory.apply(lambda x: make_lst(x))
    
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] == num
        tourn_df = df[mask]
        
        for idx, row in tourn_df.iterrows(): # go through each row of tournament 
            mask = tourn_df.index < idx
            df_prior = tourn_df[mask] # hands that occured previously in this tournament 
            
            for player in row['player_names']: # looping through all players present in hand
                player_actions = []
                
                for dic in df_prior['action_type']: # looking at all prior hands 
                    if player in dic:
                        player_actions.append(dic[player])

            df.loc[idx, 'prior_actions'] = []
            
            
            df.loc[idx, 'prior_actions'].append(player)
            df.loc[idx, 'prior_actions'].append(player_actions)

            
#     for idx, row in df.iterrows():
#         if idx in total_dic:
#             df['prior_actions'].iloc[idx] = total_dic[idx]
    return df

In [9]:
test = prior_actions_for_stats_trying_something_else(df)

/home/sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


ValueError: cannot set using a multi-index selection indexer with a different length than the value

In [18]:
def make_lst(x):
    return []





In [19]:
df

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap,prior_actions
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0,[]
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0,[]
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0,[]
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0,[]
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:25,82,41343764,5.25,Holdem NL,200.0,"[HA, H3]",1850.0,Hyper Turbo (500 Chips),700.0,1850.0,2,0.500000,6,100.0,"{'P6_773472PB': 3, 'Hero': 7}","[P6_773472PB, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,7.0,200.0,9.25,-1150.0,1.0,1,0.378378,0,0.0,0.0,0.0,1650.0,14.0,3.0,1,0,11.0,[]
54300,54334,4286799339,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 08:12:29,82,41343587,5.25,Holdem NL,100.0,"[D6, H4]",196.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P1_574120EG, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,3.0,200.0,0.98,-100.0,0.0,0,0.489796,0,0.0,0.0,0.0,0.0,6.0,4.0,0,0,2.0,[]
54301,54335,4286799383,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:34,82,41343764,5.25,Holdem NL,100.0,"[C8, H6]",700.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P6_773472PB, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,4.0,200.0,3.50,-100.0,0.0,0,0.857143,0,0.0,0.0,0.0,0.0,8.0,6.0,0,0,2.0,[]
54302,54336,4286799396,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 08:12:41,112,41343587,5.25,Holdem NL,96.0,"[DA, H7]",96.0,Hyper Turbo (500 Chips),0.0,96.0,2,0.500000,6,0.0,{},"[P1_5741

In [23]:
# save this function as is, will develop the better version seperately 
def prior_actions_for_stats(df):
    '''
    finds actions of player prior to the hand currently being played 
    '''
    mask = df['TournamentNumber'] != '' # ignores cash hands for time being, make this optional w/ argument? 
    df = df[mask]
    df['prior_actions'] = df.HandHistory.apply(lambda x: make_lst(x))
    total_dic = {} # probably replace this w/ assignment directly to the dataframe 
    unique_TN = df['TournamentNumber'].unique()
    for num in unique_TN: # iterates through all unique tournament numbers
        mask = df['TournamentNumber'] == num
        tourn_df = df[mask]
        for idx, row in tourn_df.iterrows(): # go through each row of tournament 
            row_dic = {}
            mask = tourn_df.index < idx
            df_prior = tourn_df[mask] # hands that occured previously in this tournament 
            for player in row['player_names']: # looping through all players present in hand
                player_actions = []
                for dic in df_prior['action_type']: # looking at all prior hands 
                    if player in dic:
                        player_actions.append(dic[player])
                row_dic[player] = player_actions
            df.loc[idx, 'prior_actions'].append(row_dic)

    return df


In [24]:
test = prior_actions_for_stats(df)

/home/sam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
test

,HandHistoryId,HandNumber,PokerSiteId,HandHistory,HandHistoryTimestamp,GameType,TournamentNumber,buyin,cash_or_tourn,my_blind_anti_total,my_cards,starting_stack,tournament_type,won,bet,total_players,position,table_max_players,bets_before_my_preflop_action,action_type,player_names,hour,year_month_day,days_since_start,hand_frequency,blinds,made_money,card_rank,BB,BB_in_stack,net_outcome,all_in,money_beyond_blind,outcome_relative_to_start,made_or_lost,flop_bet,turn_bet,river_bet,preflop_bet,high_card,low_card,suited,pocket_pair,gap,prior_actions
0,1,3951210633,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:08:08,1,459742,1.10,Holdem NL,20.0,"[C5, H2]",1500.0,3-Table Sit & Go,0.0,20.0,9,0.222222,9,80.0,"{'P26_251842VD': 3, 'P4_131206CX': 0, 'P11_924...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,0.5,20.0,75.00,-20.0,0.0,0,0.986667,0,0.0,0.0,0.0,0.0,5.0,2.0,0,0,3.0,"[{'P18_572249IY': [], 'P14_459713MR': [], 'P25..."
1,2,3951211988,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:09:34,1,459742,1.10,Holdem NL,10.0,"[C9, S9]",1480.0,3-Table Sit & Go,640.0,420.0,9,0.111111,9,40.0,"{'P4_131206CX': 0, 'P11_924370QJ': 0, 'P18_572...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",1,9.0,20.0,74.00,220.0,0.0,1,1.148649,1,320.0,0.0,0.0,90.0,9.0,9.0,0,1,0.0,"[{'P18_572249IY': [0], 'P14_459713MR': [0], 'P..."
2,3,3951212830,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:11:47,1,459742,1.10,Holdem NL,0.0,"[S3, DK]",1700.0,3-Table Sit & Go,0.0,0.0,9,1.000000,9,200.0,"{'P11_924370QJ': 0, 'P18_572249IY': 0, 'P14_45...","[P18_572249IY, P14_459713MR, P25_871309MO, P3_...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,3.0,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,13.0,3.0,0,0,10.0,"[{'P18_572249IY': [0, 0], 'P14_459713MR': [0, ..."
3,4,3951214131,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:13:37,1,459742,1.10,Holdem NL,0.0,"[C9, D8]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.875000,9,0.0,"{'P18_572249IY': 0, 'P25_871309MO': 0, 'P3_286...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,5.5,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,9.0,8.0,0,0,1.0,"[{'P18_572249IY': [0, 0, 0], 'P25_871309MO': [..."
4,5,3951215154,1,"[<session sessioncode=""459742"">, <general>, <m...",2019-10-23 21:14:31,1,459742,1.10,Holdem NL,0.0,"[S7, HQ]",1700.0,3-Table Sit & Go,0.0,0.0,8,0.750000,9,0.0,"{'P25_871309MO': 0, 'P3_286963JX': 0, 'P5_4057...","[P18_572249IY, P25_871309MO, P3_286963JX, P5_4...",21,2019-10-23,0,2155,"[10.0, 20.0]",0,2.0,20.0,85.00,0.0,0.0,0,1.000000,1,0.0,0.0,0.0,0.0,12.0,7.0,0,0,5.0,"[{'P18_572249IY': [0, 0, 0, 0], 'P25_871309MO'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54299,54333,4286799273,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:25,82,41343764,5.25,Holdem NL,200.0,"[HA, H3]",1850.0,Hyper Turbo (500 Chips),700.0,1850.0,2,0.500000,6,100.0,"{'P6_773472PB': 3, 'Hero': 7}","[P6_773472PB, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,7.0,200.0,9.25,-1150.0,1.0,1,0.378378,0,0.0,0.0,0.0,1650.0,14.0,3.0,1,0,11.0,"[{'P6_773472PB': [0, 23, 23, 0, 0, 23, 0, 0, 2..."
54300,54334,4286799339,1,"[<session sessioncode=""4917292"">, <general>, <...",2021-03-30 08:12:29,82,41343587,5.25,Holdem NL,100.0,"[D6, H4]",196.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P1_574120EG, Hero]",8,2021-03-30,524,699,"[100.0, 200.0]",0,3.0,200.0,0.98,-100.0,0.0,0,0.489796,0,0.0,0.0,0.0,0.0,6.0,4.0,0,0,2.0,"[{'P1_574120EG': [0, 0, 23, 0, 0, 0, 23, 0, 0,..."
54301,54335,4286799383,1,"[<session sessioncode=""3408296"">, <general>, <...",2021-03-30 08:12:34,82,41343764,5.25,Holdem NL,100.0,"[C8, H6]",700.0,Hyper Turbo (500 Chips),0.0,100.0,2,1.000000,6,0.0,{'Hero': 0},"[P